In [2]:
%run dihiggs_dataset.ipynb

import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F   # NOTE: I don't think this is used
import torch.autograd as autograd
import torch

from IPython import display
from matplotlib import pyplot as plt

#
print(torch.cuda.is_available())

True


In [3]:

# os.makedirs("images", exist_ok=True)

#parser = argparse.ArgumentParser()

"""
parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--n_critic", type=int, default=5, help="number of training steps for discriminator per iter")
parser.add_argument("--clip_value", type=float, default=0.01, help="lower and upper clip value for disc. weights")
parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples")
"""
#opt = parser.parse_args()
#print(opt)
class opt():   # Class used for optimizers in the future. Defines all variables and stuff needed.
    n_epochs = 20000   # an epoch is the number of times it works through the entire training set
    batch_size = 1000   # the training set is broken up into batches, 
                        # and the average loss is used from a given batch for back propagation
    lr =  0.0002 # 0.001   # learning rate (how much to change based on error)
    b1 = 0     # 0.9 # Used for Adam. Exponential decay rate for the first moment. 
    b2 = 0.9   # 0.999 # Used for Adam. Exponential decay rate for the second moment estimates (gradient squared)
    #NOTE: The default epsilon for torch.optim.adam is 1e-8, so I will just leave it that way
    
    #n_cpu = 2   # not used rn
    latent_dim = 100 #size of noise input to generator (latent space) 
    #img_size = 28
    # channels = 1   # Only used for img_shape right below, and img_shape isn't needed
    n_critic = 5   # The generator is trained after this many critic steps
    #   clip_value = 0.01   # No other usages rn. 
    sample_interval = 400   # Determines when a to save the image(s?) generated
    
    Xi = 10;   # multiplier for recursively finding r_adversarial
    
    # Loss weight for alp penalty
    lambda_alp = 100

# img_shape = (opt.channels, opt.img_size, opt.img_size)   # Not used rn

cuda = True if torch.cuda.is_available() else False




class Generator(nn.Module):
    """
    Create hidden layers. Apply normalization. Apply leaky relu. 
    """
    def __init__(self):
        super(Generator, self).__init__()   

        def block(in_feat, out_feat, normalize=True):   # This function creates the hidden layers
            layers = [nn.Linear(in_feat, out_feat)]   # layer is a hidden layer. Takes input
                                                      # (batch_size,in_feat) and give an output (batch_size,out_feat)
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))   # adds normalization to what Layers does to input and comes out in
                                                               # size (batch_size,out_feat). I think this does bn1d(linear(input))
            layers.append(nn.LeakyReLU(0.2, inplace=True))   # inplace means just modify input, don't allocate more memory
            return layers



        """
        stores layers and functions applied to layers
        """       
        self.model = nn.Sequential(   
            *block(opt.latent_dim, 128, normalize=False),   # first layer
            *block(128, 256),   # second layer
            *block(256, 512),   # 3rd layer
            *block(512, 1024),   # 4th layer
            nn.Linear(1024, 25),   # final layer. Output is size 25
            nn.Tanh()   # Using tanh for final output (why tanh vs leaky relu?)
        )

    def forward(self, z):
        """
        applies layers to input to get img
        """
        img = self.model(z)   # applies model (layers and functions on layers) to z
        #img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    """
    Discriminator/critic layers
    """
    def __init__(self):
        super(Discriminator, self).__init__()   # Just uses the module constructor with name Discriminator 

        self.model = nn.Sequential(
            nn.Linear(25, 512),   # first layer
            nn.LeakyReLU(0.2, inplace=True),   # apply leaky relu to layer
            nn.Linear(512, 256),   # 2nd layer
            nn.LeakyReLU(0.2, inplace=True),   # apply leaky relu to layer
            nn.Linear(256, 1),   # Final layer to give output. Output is size 1 (validity score)
                                 # NOTE: weird to end with comma
        )

    def forward(self, img):
        """
        applies model to image and gives validity score
        """
        img_flat = img.view(img.shape[0], -1)   # TODO: Figure out what this does 
        validity = self.model(img_flat)   # calculates validity score
        #print("forward validity from discriminator: " + str((np.max(np.abs(validity.detach().numpy())))))
        return validity


# ******* OUT OF CLASSES NOW ************

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    
    
# Configure data loader - CHANGE
os.makedirs("./data/", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
   DiHiggsSignalMCDataset('./DiHiggs Data', generator_level = False),
   batch_size=opt.batch_size,
   shuffle=True,
)
print('done')

done


In [4]:
def compute_ALP(D, real_samples, fake_samples):   # TODO: Find out why these are .data
    """
    Calculates the gradient penalty loss for WGAN GP
    D input will be discrimantor function
    real_samples and fake_samples are from reality and generator. Both are sent in via memory location of buffer
    
    """
    
    # Random weight term for interpolation between real and fake samples (how much of each)
    alpha = Tensor(np.random.random((real_samples.size(0),1)))   # This is a tensor designating which to use where
    #print(alpha)
  #  print(alpha.shape)
    # Get random interpolation between real and fake samples
   # print(real_samples.shape)
    
    # Gets some of real and some of fake samples for gradient penalty calculation
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    # .requires grad is something attached to all tensors and can be used to speed up (by making false I think)
    # It is automatically false, but if you need gradient then set to be true
    # TODO: Understand how this statement works
    
    
    ################## CALCULATE R ADVERSARIAL ###############################################
    # start with random unit vector r0
    r0 = np.random.rand(interpolates.shape[0], interpolates.shape[1])
    r0 = Tensor(r0/r0.max(axis = 0)).requires_grad_(True)
    #print(r[0])
    
    #  add this initial r to our random data points
    interpol_y0 = (interpolates + opt.Xi * r0).requires_grad_(True)   #.requires_grad_(True)
    # run the discriminator on both of these
    d_interpolates = D(interpolates)   # Run discriminator on interpolates to get validity scores
    d_interpol_y0 = D(interpol_y0)   # do the same for the adjusted interpolates to find r adversarial

    
    # find gradient(d(f(x) - f(x+r)))
    difference = (d_interpolates - d_interpol_y0).requires_grad_(True)  #.requires_grad_(True)
    #print("d interpolates: " + str(d_interpolates.shape) + " " + str(d_interpolates.type))
    #print("difference: " + str(difference.shape) + " " + str(difference.type))
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False) 
    gradient_r0 = autograd.grad(
        outputs=difference,
        inputs=r0,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    
    # finally, find r_adversarial!
    epsilon_r = np.random.uniform(0.1,10)
    r_adv = epsilon_r * gradient_r0/np.linalg.norm(gradient_r0.cpu().detach().numpy())
    #print(np.max(np.linalg.norm(r_adv.cpu().detach().numpy())))
###########################################################################################################

######### Now find the loss ###########################
    
    interpol_adversarial = (interpolates + r_adv).requires_grad_(True)
    d_interpol_adv = D(interpol_adversarial)
    abs_difference = np.abs((d_interpolates - d_interpol_adv).cpu().detach().numpy())/ \
    (np.linalg.norm(r_adv.cpu().detach().numpy())) - 1
    squared = np.square(np.maximum(abs_difference,np.zeros(100)))
    #print("Max of alp before mean: " + str(np.max(np.abs(squared))))
    
    alp_penalty = squared.mean()
   # print("ALP final: " + str(alp_penalty))
    
    return alp_penalty


In [6]:
# Optimizers (Adam optimizers are an alternative to stochastic gradient descent. TODO learn more about them)
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))   
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

batches_done = 0   # Counter for batches
for epoch in range(opt.n_epochs):   # Loop through all epochs
    for i, x in enumerate(dataloader): # x is in dataloader (a batch I think). i
                                       # is the index of x (number of times critic is trained this epoch)

        # Configure input
        real_imgs = Variable(x.type(Tensor))   # Variable is a wrapper for the Tensor x was just made into

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()   # Make gradients zero so they don't accumulate

        # Sample noise (latent space) to make generator input
        z = Variable(Tensor(np.random.normal(0, 1, (x.shape[0], opt.latent_dim))))   # Once again Variable wraps the Tensor
#         print(type(x))
#         print(x.shape)
#         print(x[0].shape())
#         print(z.shape)

        # Generate a batch of images from the latent space sampled
        fake_imgs = generator(z)

        #print(fake_imgs[0])

        # Calculate validity score for real images
        real_validity = discriminator(real_imgs)

        # Calculate validity score for fake images
        fake_validity = discriminator(fake_imgs)

        # Calculate gradient penalty
        alp = compute_ALP(discriminator, real_imgs.data, fake_imgs.data)
        # TODO: figure out why .data is used

        # Calculate loss for critic (Adversarial loss)
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + opt.lambda_alp * alp

        d_loss.backward()   # Do back propagation 
        optimizer_D.step()   # Update parameters based on gradients for individuals

        optimizer_G.zero_grad()   # Resets gradients for generator to be zero to avoid accumulation

        # Train the generator every n_critic steps
        if i % opt.n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)

            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()


            # ----------------------------
            # Save stuff when time is right
            # ----------------------------
            if batches_done % 10 == 0:
                print(
                    "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                    % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
                )

            if batches_done % opt.sample_interval == 0:
                save_image(fake_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

            batches_done += opt.n_critic
    if epoch % 10 == 0:
        z = Variable(Tensor(np.random.normal(0, 1, (300000, opt.latent_dim))))
        fake_data = generator(z)
        np.save('./gen_data_alp_optimizer/{num_batches}.npy'.format(num_batches=batches_done), fake_data.cpu().detach().numpy())


[Epoch 0/20000] [Batch 0/192] [D loss: -3.152621] [G loss: -2.843803]
[Epoch 0/20000] [Batch 10/192] [D loss: -3.962737] [G loss: 0.911224]
[Epoch 0/20000] [Batch 20/192] [D loss: -4.523022] [G loss: 3.748593]
[Epoch 0/20000] [Batch 30/192] [D loss: -4.288134] [G loss: 5.191018]
[Epoch 0/20000] [Batch 40/192] [D loss: -3.266751] [G loss: 5.361424]
[Epoch 0/20000] [Batch 50/192] [D loss: -1.855717] [G loss: 2.868436]
[Epoch 0/20000] [Batch 60/192] [D loss: -0.640530] [G loss: -2.028757]
[Epoch 0/20000] [Batch 70/192] [D loss: -0.171665] [G loss: -7.061730]
[Epoch 0/20000] [Batch 80/192] [D loss: -0.176010] [G loss: -11.562768]
[Epoch 0/20000] [Batch 90/192] [D loss: -0.491660] [G loss: -15.133231]
[Epoch 0/20000] [Batch 100/192] [D loss: -0.413961] [G loss: -17.903133]
[Epoch 0/20000] [Batch 110/192] [D loss: -0.167496] [G loss: -18.970419]
[Epoch 0/20000] [Batch 120/192] [D loss: 0.145327] [G loss: -16.756975]
[Epoch 0/20000] [Batch 130/192] [D loss: -0.007465] [G loss: -12.080933]
[Ep

[Epoch 5/20000] [Batch 185/192] [D loss: 0.014165] [G loss: 6.298416]
[Epoch 6/20000] [Batch 0/192] [D loss: 0.106718] [G loss: 6.926940]
[Epoch 6/20000] [Batch 10/192] [D loss: -0.010612] [G loss: 7.204154]
[Epoch 6/20000] [Batch 20/192] [D loss: -0.127037] [G loss: 7.116861]
[Epoch 6/20000] [Batch 30/192] [D loss: -0.194945] [G loss: 7.611331]
[Epoch 6/20000] [Batch 40/192] [D loss: -0.174652] [G loss: 7.616408]
[Epoch 6/20000] [Batch 50/192] [D loss: -0.091590] [G loss: 8.456738]
[Epoch 6/20000] [Batch 60/192] [D loss: -0.058246] [G loss: 7.389418]
[Epoch 6/20000] [Batch 70/192] [D loss: -0.024537] [G loss: 6.906200]
[Epoch 6/20000] [Batch 80/192] [D loss: -0.114267] [G loss: 9.969329]
[Epoch 6/20000] [Batch 90/192] [D loss: -0.108860] [G loss: 7.030690]
[Epoch 6/20000] [Batch 100/192] [D loss: -0.202070] [G loss: 8.392245]
[Epoch 6/20000] [Batch 110/192] [D loss: -0.191989] [G loss: 7.230799]
[Epoch 6/20000] [Batch 120/192] [D loss: -0.165171] [G loss: 6.022962]
[Epoch 6/20000] [Ba

[Epoch 11/20000] [Batch 175/192] [D loss: -0.265709] [G loss: 13.584658]
[Epoch 11/20000] [Batch 185/192] [D loss: -0.289104] [G loss: 14.226882]
[Epoch 12/20000] [Batch 0/192] [D loss: -0.122004] [G loss: 14.035115]
[Epoch 12/20000] [Batch 10/192] [D loss: -0.243320] [G loss: 14.731155]
[Epoch 12/20000] [Batch 20/192] [D loss: -0.373304] [G loss: 14.895585]
[Epoch 12/20000] [Batch 30/192] [D loss: -0.375488] [G loss: 14.018515]
[Epoch 12/20000] [Batch 40/192] [D loss: -0.438274] [G loss: 15.049822]
[Epoch 12/20000] [Batch 50/192] [D loss: -0.445325] [G loss: 14.515594]
[Epoch 12/20000] [Batch 60/192] [D loss: -0.451390] [G loss: 15.339748]
[Epoch 12/20000] [Batch 70/192] [D loss: -0.326023] [G loss: 18.505610]
[Epoch 12/20000] [Batch 80/192] [D loss: -0.261873] [G loss: 16.563185]
[Epoch 12/20000] [Batch 90/192] [D loss: -0.336485] [G loss: 13.785924]
[Epoch 12/20000] [Batch 100/192] [D loss: -0.339668] [G loss: 17.751259]
[Epoch 12/20000] [Batch 110/192] [D loss: -0.310640] [G loss: 

[Epoch 17/20000] [Batch 145/192] [D loss: -0.315018] [G loss: 20.600071]
[Epoch 17/20000] [Batch 155/192] [D loss: -0.251581] [G loss: 24.205770]
[Epoch 17/20000] [Batch 165/192] [D loss: -0.237000] [G loss: 24.030228]
[Epoch 17/20000] [Batch 175/192] [D loss: -0.157095] [G loss: 24.287806]
[Epoch 17/20000] [Batch 185/192] [D loss: -0.277369] [G loss: 23.630232]
[Epoch 18/20000] [Batch 0/192] [D loss: -0.056885] [G loss: 24.908924]
[Epoch 18/20000] [Batch 10/192] [D loss: -0.266895] [G loss: 20.828655]
[Epoch 18/20000] [Batch 20/192] [D loss: -0.082745] [G loss: 21.318645]
[Epoch 18/20000] [Batch 30/192] [D loss: -0.237598] [G loss: 25.413958]
[Epoch 18/20000] [Batch 40/192] [D loss: -0.118542] [G loss: 22.081951]
[Epoch 18/20000] [Batch 50/192] [D loss: -0.315563] [G loss: 21.157564]
[Epoch 18/20000] [Batch 60/192] [D loss: -0.270634] [G loss: 24.146811]
[Epoch 18/20000] [Batch 70/192] [D loss: -0.227907] [G loss: 24.265709]
[Epoch 18/20000] [Batch 80/192] [D loss: -0.210028] [G loss:

[Epoch 23/20000] [Batch 115/192] [D loss: -0.257866] [G loss: 24.824190]
[Epoch 23/20000] [Batch 125/192] [D loss: -0.259542] [G loss: 30.094696]
[Epoch 23/20000] [Batch 135/192] [D loss: -0.372759] [G loss: 29.601370]
[Epoch 23/20000] [Batch 145/192] [D loss: -0.421480] [G loss: 28.209476]
[Epoch 23/20000] [Batch 155/192] [D loss: -0.388191] [G loss: 32.323616]
[Epoch 23/20000] [Batch 165/192] [D loss: -0.445690] [G loss: 25.871849]
[Epoch 23/20000] [Batch 175/192] [D loss: -0.328360] [G loss: 31.533752]
[Epoch 23/20000] [Batch 185/192] [D loss: -0.258480] [G loss: 28.409630]
[Epoch 24/20000] [Batch 0/192] [D loss: -0.084116] [G loss: 27.950993]
[Epoch 24/20000] [Batch 10/192] [D loss: -0.394966] [G loss: 30.014606]
[Epoch 24/20000] [Batch 20/192] [D loss: -0.313253] [G loss: 31.300846]
[Epoch 24/20000] [Batch 30/192] [D loss: -0.171110] [G loss: 30.611427]
[Epoch 24/20000] [Batch 40/192] [D loss: -0.361588] [G loss: 29.418724]
[Epoch 24/20000] [Batch 50/192] [D loss: -0.326279] [G lo

...

[Epoch 240/20000] [Batch 10/192] [D loss: 4764.572266] [G loss: 53.431484]
[Epoch 240/20000] [Batch 20/192] [D loss: 6647.803223] [G loss: 65.718163]
[Epoch 240/20000] [Batch 30/192] [D loss: 5534.676270] [G loss: 55.841427]
[Epoch 240/20000] [Batch 40/192] [D loss: 4471.137695] [G loss: 63.081432]
[Epoch 240/20000] [Batch 50/192] [D loss: 5661.634277] [G loss: 81.974037]
[Epoch 240/20000] [Batch 60/192] [D loss: 4583.225586] [G loss: 41.122448]
[Epoch 240/20000] [Batch 70/192] [D loss: 4532.322754] [G loss: 82.053299]
[Epoch 240/20000] [Batch 80/192] [D loss: 7387.117188] [G loss: 54.530224]
[Epoch 240/20000] [Batch 90/192] [D loss: 4106.337891] [G loss: 85.963226]
[Epoch 240/20000] [Batch 100/192] [D loss: 4464.551270] [G loss: 41.633926]
[Epoch 240/20000] [Batch 110/192] [D loss: 3896.030273] [G loss: 80.513573]
[Epoch 240/20000] [Batch 120/192] [D loss: 4705.818359] [G loss: 50.018154]
[Epoch 240/20000] [Batch 130/192] [D loss: 3758.879639] [G loss: 74.581306]
[Epoch 240/20000] [Ba

[Epoch 743/20000] [Batch 115/192] [D loss: 11413.862305] [G loss: 142.069000]
[Epoch 743/20000] [Batch 125/192] [D loss: 10231.112305] [G loss: 148.501205]
[Epoch 743/20000] [Batch 135/192] [D loss: 35778.867188] [G loss: 125.409554]
[Epoch 743/20000] [Batch 145/192] [D loss: 22625.380859] [G loss: 148.396851]
[Epoch 743/20000] [Batch 155/192] [D loss: 20142.076172] [G loss: 149.626633]
[Epoch 743/20000] [Batch 165/192] [D loss: 28534.707031] [G loss: 126.482643]
[Epoch 743/20000] [Batch 175/192] [D loss: 25782.294922] [G loss: 141.317535]
[Epoch 743/20000] [Batch 185/192] [D loss: 13126.467773] [G loss: 147.212097]
[Epoch 744/20000] [Batch 0/192] [D loss: 20531.011719] [G loss: 138.374100]
[Epoch 744/20000] [Batch 10/192] [D loss: 25877.210938] [G loss: 145.290207]
[Epoch 744/20000] [Batch 20/192] [D loss: 25320.097656] [G loss: 144.169022]
[Epoch 744/20000] [Batch 30/192] [D loss: 10867.386719] [G loss: 141.201401]
[Epoch 744/20000] [Batch 40/192] [D loss: 10807.400391] [G loss: 145.

[Epoch 749/20000] [Batch 5/192] [D loss: 16163.678711] [G loss: 143.356308]
[Epoch 749/20000] [Batch 15/192] [D loss: 27783.796875] [G loss: 138.644760]
[Epoch 749/20000] [Batch 25/192] [D loss: 11645.258789] [G loss: 155.906296]
[Epoch 749/20000] [Batch 35/192] [D loss: 25330.824219] [G loss: 116.987709]
[Epoch 749/20000] [Batch 45/192] [D loss: 35532.121094] [G loss: 115.350395]
[Epoch 749/20000] [Batch 55/192] [D loss: 26816.031250] [G loss: 149.352539]
[Epoch 749/20000] [Batch 65/192] [D loss: 13816.340820] [G loss: 158.773529]
[Epoch 749/20000] [Batch 75/192] [D loss: 25645.703125] [G loss: 142.061935]
[Epoch 749/20000] [Batch 85/192] [D loss: 27209.050781] [G loss: 122.414825]
[Epoch 749/20000] [Batch 95/192] [D loss: 17384.208984] [G loss: 148.733734]
[Epoch 749/20000] [Batch 105/192] [D loss: 16956.693359] [G loss: 149.010437]
[Epoch 749/20000] [Batch 115/192] [D loss: 17592.468750] [G loss: 138.105255]
[Epoch 749/20000] [Batch 125/192] [D loss: 14451.923828] [G loss: 131.28044

[Epoch 754/20000] [Batch 90/192] [D loss: 3041.360352] [G loss: 131.407394]
[Epoch 754/20000] [Batch 100/192] [D loss: 7060.447266] [G loss: 149.581909]
[Epoch 754/20000] [Batch 110/192] [D loss: 35059.386719] [G loss: 129.485535]
[Epoch 754/20000] [Batch 120/192] [D loss: 24746.437500] [G loss: 141.417389]
[Epoch 754/20000] [Batch 130/192] [D loss: 30165.169922] [G loss: 142.295853]
[Epoch 754/20000] [Batch 140/192] [D loss: 28117.468750] [G loss: 137.985031]
[Epoch 754/20000] [Batch 150/192] [D loss: 32388.347656] [G loss: 133.879913]
[Epoch 754/20000] [Batch 160/192] [D loss: 24407.554688] [G loss: 147.216095]
[Epoch 754/20000] [Batch 170/192] [D loss: 20638.605469] [G loss: 137.108139]
[Epoch 754/20000] [Batch 180/192] [D loss: 21600.615234] [G loss: 149.526794]
[Epoch 754/20000] [Batch 190/192] [D loss: 24406.835938] [G loss: 127.959831]
[Epoch 755/20000] [Batch 5/192] [D loss: 12850.659180] [G loss: 159.449127]
[Epoch 755/20000] [Batch 15/192] [D loss: 20803.687500] [G loss: 141.

KeyboardInterrupt: 